# STINTSY Technical Document

### Group FOP:
- Keil Christopher Finez
- Azriel Matthew Ortega
- James Andrew Perez

## Description of the Dataset and Task
- An overview or description of the data is provided, including how it was collected, and its implications on the types of conclusions that could be made from the data. A description of the variables, observations, and/or structure of the data is provided. The target task is well introduced and clearly defined.

## Exploratory data analysis
-  Appropriate summaries and visualizations are presented. Insights into how the EDA can help the model training is mentioned.

## Data pre-processing and cleaning
 - If no preprocessing or cleaning is done, there is a justification on why it was not needed.

## Model Training
- The appropriate models are used to accomplish the machine learning task. Justification of choosing the models is shown.

## Model selection and hyperparameter tuning
- Appropriate data-driven error analysis is made, and changes to the model selection and hyperparameters are performed to improve model performance. The study exhausts improvements that can be done to the model